# Working with databases

## DB-API 2.0

The Python Database API Specification, often referred to as DB-API, is a standard interface for connecting to relational databases from Python programs. It provides a consistent and uniform way to interact with databases, regardless of the underlying database management system (DBMS) being used. DB-API 2.0 was introduced to improve and standardize database connectivity in Python, making it easier for developers to work with databases in a consistent manner.

## sqlite3

SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle.

The `sqlite3` module in Python Standard Library implements DB-API 2.0 for SQlite databases.

To work with a database we have to either create a database or use an existing one. Call `sqlite3.connect(`) to create a connection to the database `tutorial.db` in the current working directory, implicitly creating it if it does not exist:


In [1]:
import sqlite3
con = sqlite3.connect("tutorial.db")

The returned `Connection` object `con` represents the connection to the on-disk database.

In order to execute SQL statements and fetch results from SQL queries, we will need to use a database cursor. Call `con.cursor()` to create the `Cursor`:

In [2]:
cur = con.cursor()

Now that we’ve got a database connection and a cursor, we can create a database table `movie` with columns for title, release year, and review score.

In [3]:
cur.execute("CREATE TABLE movie(title TEXT, year INTEGER, score INTEGER)")

Now, add two rows of data supplied as SQL literals by executing an INSERT statement, once again by calling `cur.execute()`:

In [4]:
cur.execute("""
    INSERT INTO movie VALUES
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5)
""")

The `INSERT` statement implicitly opens a transaction, which needs to be committed before changes are saved in the database. Call `con.commit()` on the connection object to commit the transaction:

In [5]:
con.commit()

We can verify that the data was inserted correctly by executing a `SELECT` query.

In [6]:
res = cur.execute("SELECT score FROM movie")
res.fetchall()

[(8.2,), (7.5,)]

Now, insert three more rows by calling `cur.executemany()`:

In [7]:
data = [
    ("Monty Python Live at the Hollywood Bowl", 1982, 7.9),
    ("Monty Python's The Meaning of Life", 1983, 7.5),
    ("Monty Python's Life of Brian", 1979, 8.0),
]
cur.executemany("INSERT INTO movie VALUES(?, ?, ?)", data)
con.commit()  # Remember to commit the transaction after executing INSERT.

Notice that `?` placeholders are used to bind data to the query. Always use placeholders instead of string formatting to bind Python values to SQL statements, to avoid SQL injection attacks.

We can verify that the new rows were inserted by executing a SELECT query, this time iterating over the results of the query:

In [8]:
for row in cur.execute("SELECT year, title FROM movie ORDER BY year"):
    print(row)

(1971, 'And Now for Something Completely Different')
(1975, 'Monty Python and the Holy Grail')
(1979, "Monty Python's Life of Brian")
(1982, 'Monty Python Live at the Hollywood Bowl')
(1983, "Monty Python's The Meaning of Life")


Finally, close the cursor and the database connection:

In [10]:
cur.close()
con.close()

## Other DBMS

DB-API 2.0 has been widely adopted by Python database libraries and modules, making it possible to work with various databases using a consistent programming interface. Developers can choose from a variety of database drivers and modules that implement this standard to work with their preferred DBMS.

- [Oracle](https://python-oracledb.readthedocs.io/en/latest/)
- [MySQL](https://dev.mysql.com/doc/connector-python/en/)
- [PostgrsSQL](https://www.psycopg.org/docs/)
- SQL Server: [pyodbc](https://github.com/mkleehammer/pyodbc/wiki) / [cTDS](https://zillow.github.io/ctds/)

## Exercises

1. Try connecting to an existing database and run a few queries. Parametrize the queries using data read from a file.
1. Create a new table `book` with three fields: `title`, `author`, `year`. Insert multiple rows in the table (use `executemany`). Select all books between two given years and write the result to a file.